In [61]:
from random import uniform as randFloat
from tqdm import tqdm_notebook as tqdm
from statistics import mean
import ipyvolume.pylab as ipv
import numpy as np
import time
import math

In [100]:
class Bird():
    def __init__(self, id, flock, position, velocity = [0, 0, 0]):
        self.position = position
        self.velocity = velocity
        self.nearestneighbors = []
        self.flock = flock
        self.id = id
        
    def nearme(self):
        """
        determines 7 nearest neighbors (except right behind) ** still looks all around not everywhere but right behind
        """
        #close = math.sqrt(math.pow(self.flock.xmax - self.flock.xmin, 2), math.pow(self.flock.ymax - self.flock.ymin, 2), math.pow(self.flock.zmax - self.flock.zmin, 2))
        close = 999999
        closest = [close, close, close, close, close, close, close]
        closestbirds = [0, 0, 0, 0, 0, 0, 0]
        for i in range(7):
            for bird in range(self.flock.size):
                if (bird != self.id):
                    distance = math.sqrt(math.pow(self.position[0] - self.flock.birds[bird].position[0], 2) + math.pow(self.position[1] - self.flock.birds[bird].position[1], 2) + math.pow(self.position[2] - self.flock.birds[bird].position[2], 2))
                    if (bird not in closestbirds):
    #                     if (not self.behindme(self.flock.birds[bird])):
                            if (distance < closest[i]):
                                closestbirds[i] = bird
                                closest[i] = distance
        self.nearestneighbors.append(closestbirds)
        return closestbirds
    
### STILL WORKING ON ###
#     def behindme(self, bird):
#         xdistance, ydistance, zdistance = (bird.position[0] - self.position[0]), (bird.position[1] - self.position[1]), (bird.position[2] - self.position[2])
#         xlim, ylim, zlim = [self.position[0], self.velocity[0]*self.position[0]*(self.flock.xmax - self.flock.xmin)], [self.position[1], self.velocity[1]*self.position[1]*(self.flock.ymax - self.flock.ymin)], [self.position[2], self.velocity[2]*self.position[2]*(self.flock.zmax - self.flock.zmin)]
#         if ( xlim[0] > bird.position[0] > xlim[1]):
#             return True
#         else:
#             return False
        
    def align(self, variability = 1):
        """
        Aligns itself by determining velocity based on 7 nearest neighbors
        """
        output = [0, 0, 0]
        self.nearme()
        for j in range(3):
            output[j] = sum(self.flock.birds[self.nearestneighbors[0][i]].velocity[j] for i in range(7))/len(self.nearestneighbors[0]) + randFloat(-variability, variability)
        return output

    def separate(self):
        xforces, yforces, zforces = [], [], []
        for bird in self.nearestneighbors[0]:
            distancex, distancey, distancez = self.position[0] - self.flock.birds[bird].position[0], self.position[1] - self.flock.birds[bird].position[1], self.position[2] - self.flock.birds[bird].position[2]
            xforce, yforce, zforce = 3/distancex, 3/distancey, 3/distancez
            xforces.append(xforce)
            yforces.append(yforce)
            zforces.append(zforce)
        output = [mean(xforces), mean(yforces), mean(zforces)]
        return output
    
    def wallforce(self):
        output = [0, 0, 0]
        return output
    
    def stick(self):
        output = [0, 0, 0]
        return output
    
    def applyForce(self, initial, force, variability, weight, interval):
        output = initial
        for i in range(3):
            output[i] = output[i] + weight*(force[i]*interval) + randFloat(-variability/2, variability/2)
        return output
    
    def refreshVelocity(self, variability, sepWeight, stickWeight, interval):
        """
        Combines multiple forces to output final velocity
        """
        output = self.align(variability)
        output = self.applyForce(initial = output, force = self.separate(), variability = variability, weight = sepWeight, interval = interval)
        #output = self.applyForce(output, self.stick(), variablility, stickWeight, interval)
        #output = self.applyForce(output, self.wallforce(), interval)
        self.velocity = output
        
        
        

In [101]:
class Flock():
    def __init__(self, xmin = 0, xmax = 100, ymin = 0, ymax = 100, zmin = 0, zmax = 100):
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax
        self.zmin = zmin
        self.zmax = zmax
        self.birds = []
        self.size = 0
        
    def center(self):
        center = [0, 0, 0]
        if (self.size == 0):
            center = [(self.xmax - self.xmin)/2, (self.ymax - self.xmin)/ 2, (self.xmax - self.xmin)/2]
        else:  
            for i in range(self.size):
                center[0] += self.birds[i].position[0]
                center[1] += self.birds[i].position[1]
                center[2] += self.birds[i].position[2]
            center[0] = center[0] / self.size
            center[1] = center[1] / self.size
            center[2] = center[2] / self.size
        return center
    
    def distance(self, bird1, bird2):
        distance = math.sqrt(math.pow(bird1.position[0] - bird2.position[0], 2) + math.pow(bird1.position[1] - bird2.position[1], 2) + math.pow(bird1.position[2] - bird2.position[2], 2))
        return distance
        
    def boundries(self):
        return [self.xmin, self.xmax, self.ymin, self.ymax, self.zmin, self.zmax]
        
    def generate(self, size = 100, velocityMin = -10, velocityMax = 10):
        for i in range(size):
            x, y, z = randFloat(self.xmin, self.xmax), randFloat(self.ymin, self.ymax), randFloat(self.zmin, self.zmax)
            u, w, v = randFloat(velocityMin, velocityMax), randFloat(velocityMin, velocityMax), randFloat(velocityMin, velocityMax)
            newBird = Bird(id = i, flock = self, position = [x, y, z], velocity = [u, w, v])
            self.birds.append(newBird)
            self.size = len(self.birds)
            del newBird
            
    def plot(self):
        fig = ipv.figure()
        flockx, flocky, flockz = np.array([]), np.array([]), np.array([])
        flocku, flockv, flockw = np.array([]), np.array([]), np.array([])
        for bird in range(self.size):
            flockx = np.append(flockx, self.birds[bird].position[0])
            flocky = np.append(flocky, self.birds[bird].position[1])
            flockz = np.append(flockz, self.birds[bird].position[2])
            flocku = np.append(flocku, self.birds[bird].velocity[0])
            flockv = np.append(flockv, self.birds[bird].velocity[1])
            flockw = np.append(flockw, self.birds[bird].velocity[2])
        centerx, centery, centerz = np.array([self.center()[0]]), np.array([self.center()[1]]), np.array([self.center()[1]])
        ipv.clear()
        flock = ipv.quiver(flockx, flocky, flockz, flocku, flockv, flockw, color='black', size = 1.5)
        center = ipv.scatter(centerx, centery, centerz, marker = 'sphere', color = 'blue', size = 2)
        ipv.style.axes_off()
        ipv.style.box_off()
        ipv.show()

    def refreshVelocities(self, variability, sepWeight = 1.0, stickWeight = 1.0, interval = .5):
        for bird in self.birds:
            bird.refreshVelocity(variability = variability, sepWeight = sepWeight, stickWeight = stickWeight, interval = interval)
    
    def __str__(self):
        return ' '.join(str(e) for e in self.birds)
    
########################
#         TEST         #
########################

flock = Flock()
flock.generate(size = 1000)

print('The flock center is: ', flock.center())

flock.plot()

#### PLOT CONNECTIONS ####
for mainbird in tqdm(range(5), unit = 'birds'): 
    for bird in flock.birds[mainbird].nearme():
        x1, x2 = flock.birds[mainbird].position[0], flock.birds[bird].position[0]
        y1, y2 = flock.birds[mainbird].position[1], flock.birds[bird].position[1]
        z1, z2 = flock.birds[mainbird].position[2], flock.birds[bird].position[2]
        x, y, z = np.array([x1, x2]), np.array([y1, y2]), np.array([z1, z2])
        ipv.plot(x, y, z, size = .1)
        time.sleep(.01)
    time.sleep(.1)


flock.refreshVelocities(1.5)
flock.plot()

#### PLOT CONNECTIONS ####
for mainbird in tqdm(range(5), unit = 'birds'): 
    for bird in flock.birds[mainbird].nearme():
        x1, x2 = flock.birds[mainbird].position[0], flock.birds[bird].position[0]
        y1, y2 = flock.birds[mainbird].position[1], flock.birds[bird].position[1]
        z1, z2 = flock.birds[mainbird].position[2], flock.birds[bird].position[2]
        x, y, z = np.array([x1, x2]), np.array([y1, y2]), np.array([z1, z2])
        ipv.plot(x, y, z, size = .1)
        time.sleep(.01)
    time.sleep(.1)

print("Completed")

# ### FIND NEARBIRDS IN ALL OF FLOCK ###
# for bird in range(2):
#     nearBirds = flock.birds[bird].nearme()
#     print(("Bird {:3}'s nearest birds are: {}").format(bird, nearBirds))
# ## PROOF THAT IT RANKS BIRDS BY CLOSENESS ###
#     for closebird in nearBirds:
#         print(flock.distance(flock.birds[bird], flock.birds[closebird]))


The flock center is:  [48.367335649877134, 50.2311558081261, 50.019378992040636]


Completed
